# Tensor equations

A numpy reimplementation of [hTensor](http://dis.um.es/profesores/alberto/material/hTensor.pdf). See also [Multilinear Algebra for Visual Geometry](http://dis.um.es/profesores/alberto/material/mvigro-td.pdf)

In [ ]:
import sympy
import numpy as np
import numpy.linalg as la

from umucv.tensor import T, mul, mapAt

def info(x):
    print(x.A.shape,x.idx)

def sht(t):
    return Matrix(t.A.tolist())

def delta(n):
    return T(np.eye(n))

## Simple example

In [ ]:
i,j,k,l,p,q = 2,3,4,5,6,7
a = T(np.random.randn(i*j*k*l).reshape(i,j,k,l),'ijkl')
x = T(np.random.randn(p*q*k*l).reshape(p,q,k,l),'pqkl')

b = a @ x

info(a)
info(x)
info(b)

s = b / a

info(s)

print(la.norm((a @ s - b).A))

In [ ]:
M1 = np.matrix('1 2 3; 4 5 7').A
M2 = np.matrix('2 1 1; 1 -1 5').A

In [ ]:
M = T(np.array([M1,M2]),'cvw')

M

In [ ]:
X = T(np.array([7,0,7]),'w')

X

In [ ]:
v = M @ X

v

In [ ]:
v / M

In [ ]:
X = T(np.array([[7,0,7],[3,3,1]]),'nw')

X

In [ ]:
v = M @ X

v

In [ ]:
v / M

## Fundamental matrix

In [ ]:
v1s = '''
 0.131250  -0.321875
-0.046875  -0.225000
-0.206250  -0.121875
-0.353125  -0.043750
 0.156250  -0.200000
-0.037500  -0.100000
-0.215625   0.006250
-0.356250   0.093750
 0.187500  -0.040625
-0.012500   0.068750
-0.215625   0.168750
-0.381250   0.250000
 0.237500   0.159375
 0.009375   0.256250
-0.218750   0.359375
-0.396875   0.446875
 0.362500   0.290625
 0.137500   0.393750
-0.062500   0.478125
-0.234375   0.550000
 0.462500   0.415625
 0.256250   0.490625
 0.056250   0.578125
-0.109375   0.628125
 0.543750   0.496875
 0.343750   0.568750
 0.146875   0.634375
 0.006250   0.684375
'''

v2s ='''
 0.425000  -0.496875
 0.293750  -0.556250
 0.134375  -0.606250
-0.056250  -0.687500
 0.465625  -0.368750
 0.325000  -0.418750
 0.146875  -0.481250
-0.034375  -0.537500
 0.515625  -0.209375
 0.365625  -0.253125
 0.181250  -0.318750
-0.012500  -0.368750
 0.568750  -0.034375
 0.403125  -0.078125
 0.212500  -0.137500
-0.003125  -0.184375
 0.418750   0.068750
 0.271875   0.040625
 0.090625  -0.000000
-0.115625  -0.046875
 0.293750   0.162500
 0.159375   0.137500
-0.021875   0.106250
-0.209375   0.062500
 0.196875   0.218750
 0.065625   0.196875
-0.100000   0.171875
-0.278125   0.143750
'''

import cv2
from io import StringIO
from umucv.htrans import homog, inhomog

v1 = T(homog(np.loadtxt(StringIO(v1s))),'ni')
v2 = T(homog(np.loadtxt(StringIO(v2s))),'nj')

In [ ]:
v1

In [ ]:
v2

In [ ]:
D = v1 * v2

D.reorder('nij')

In [ ]:
from umucv.tensor import nullTensor

F = nullTensor(D,'ji')

F

In [ ]:
D @ F

In [ ]:
D @ F.reorder('ji')

In [ ]:
D @ F('ji')

## Homography estimation

In [ ]:
x = v1
h = np.array([[1,2,3],
              [4,5,6],
              [1,1,1]])
y = T(homog(inhomog(v1.A @ h.T)), 'nj')

In [ ]:
y

In [ ]:
from umucv.tensor import eps3

In [ ]:
D = x * y

H = nullTensor(D @ eps3('ikl'), 'kj')

H

In [ ]:
H.A / H.A[2,2]

In [ ]:
H = nullTensor(D @ eps3('jkl'), 'ki')

H

In [ ]:
H.A / H.A[2,2]

## Triangulation

Interesting example of diagonal extraction, or zipping of structures.

In [ ]:
cam1 = np.array(
 [[ 512.,    0.,  320.,    0.],
  [   0.,  512.,  240.,    0.],
  [   0.,    0.,    1.,    0.]])

cam2 = np.array(
[[  27.505, -275.084,  536.765, -352.55 ],
 [  83.161,  429.114,  358.733,    4.747],
 [  -0.711,    0.118,    0.693,    0.365]])

In [ ]:
C = T(np.array([cam1,cam2]),'cvw')

with np.printoptions(precision=2, suppress=True, threshold=5):
    print(C)

In [ ]:
p3d = np.array(
    [[-0.11067118,  0.23208958,  1.22959005],
    [ 0.02605047,  0.15929016,  1.24963072],
    [ 0.15278893,  0.07876401,  1.27626139],
    [ 0.28030864,  0.01599885,  1.32861132],
    [-0.11708948,  0.12661194,  1.11779301],
    [ 0.01689746,  0.05494264,  1.14177591],
    [ 0.14953718, -0.02034495,  1.17971911],
    [ 0.26161823, -0.08734784,  1.21791671],
    [-0.12340494,  0.01364562,  0.99764912],
    [ 0.00155168, -0.05596433,  1.02972522],
    [ 0.1373791 , -0.12411526,  1.06717139],
    [ 0.2536887 , -0.18816734,  1.10157925],
    [-0.1350337 , -0.09502867,  0.88719839],
    [-0.01000757, -0.15792879,  0.92912773],
    [ 0.12754962, -0.22443005,  0.9629846 ],
    [ 0.24391027, -0.29002987,  1.003195  ],
    [-0.2249506 , -0.18288471,  0.98481174],
    [-0.08823046, -0.25155878,  1.00771513],
    [ 0.03940784, -0.31658844,  1.04822339],
    [ 0.15922148, -0.37992148,  1.09645965],
    [-0.31134116, -0.28020144,  1.07610716],
    [-0.1769637 , -0.33922077,  1.09961827],
    [-0.03994644, -0.41138884,  1.13923163],
    [ 0.08090013, -0.46762971,  1.18893698],
    [-0.39645445, -0.35924587,  1.17213933],
    [-0.25438077, -0.41932229,  1.19260766],
    [-0.11169695, -0.48399993,  1.22616359],
    [-0.00422111, -0.5485228 ,  1.28709172]])

In [ ]:
#views = C @ T(homog(p3d[[0,1],:]),'nw')
#views

In [ ]:
views = C @ T(homog(p3d[0]),'w')
views = T(homog(np.random.rand(2,2)+inhomog(views.A)),'cv')

In [ ]:
views

In [ ]:
p3d[0]

In [ ]:
X =  T(homog(p3d[0]),'w')

In [ ]:
C @ X

In [ ]:
C @ X @ eps3('vij')

In [ ]:
C @ X @ eps3('vij')  @ views('qi')

In [ ]:
mul(C @ X @ eps3('vij')  , views('ci'), sum='i')

In [ ]:
mul(C @ eps3('vij')  , views('ci'), 'i')

In [ ]:
nullTensor( mul(C @ eps3('vij')  , views('ci'), 'i'), 'w')

In [ ]:
inhomog(_.A)

In [ ]:
p3d[0]

In [ ]:
nullTensor(C @ eps3('vij')  @ views('ci'), 'w')

In [ ]:
inhomog(_.A)

In [ ]:
C

In [ ]:
mapAt(lambda c: c @ X, C ,'c')

In [ ]:
C @ X

## Exterior product and dual

In [ ]:
(T([1,0,0]) ^ T([0,1,0])) @ eps3

In [ ]:
from umucv.tensor import eps4

In [ ]:
eps4

In [ ]:
p = T([1,0,0,1])
q = T([0,1,0,1])
l = p ^ q

In [ ]:
l

In [ ]:
l @ eps4

In [ ]:
(l @ eps4) @ eps4

In [ ]:
eps4 @ eps4

In [ ]:
eps4 @ eps4('jklb')